**<h1>Mikomarket</h1>**

<h1>Wstęp</h1>
Twój znajomy Marek Adamczyk, pasjonat polityki i amatorski inwestor, postanowił "zainwestować" na Mikomarket, nowo otwartym rynku prognoz, przewidując wyniki najbliższych wyborów. Zorientował się jednak, że jednym z znaczących wskaźników są tweety znanych polityków — ich treści momentalnie wpływają na kursy zakładów i giełdy opinii. Niestety, ręczne monitorowanie tych tweetów to wyścig z czasem, a jego umiejętności programistyczne ograniczyły się do stworzenia scrapera, który potrafi pobrać tylko treść tweeta, bez wskazania autora.

Rynek jest bezlitosny, a poświęcenie kilku sekund, aby sprawdzić czy dany post jest autorstwa Korwina, czy Żukowskiej, może kosztować fortunę. Dlatego X przyszedł do Ciebie — swojego zaufanego znajomego-informatyka, z propozycją nie do odrzucenia. Prosi o stworzenie modelu, który w mgnieniu oka przypisze tweet do odpowiedniego polityka, a jako wynagrodzenie zaproponował część swoich zarobków. Pomożesz mu?

# <h2>Dostarczone pliki</h2>

- `train.csv` - Dane treningowe: zawiera treści tweetów oraz ich autorów.
- `test.csv` - Dane testowe: analogiczne do `train.csv`, ale bez oznaczonych autorów (wykorzystasz je do predykcji).
- `przykodp.csv` - Przykładowy plik w formacie w jakim ma być `odpowiedzi.csv`.
- `Zadanie.ipynb` - Notebook, który pomoże Ci rozpocząć pracę nad modelem.


# <h2>Twoje zadanie</h2>

### **1. Stworzenie modelu:**

- Wykorzystaj model allegro/herbert-base-cased.
- Zaimplementuj klasyfikator oparty na HerBERT, który przypisze tweet do jednego z pięciu autorów.
- Wybierz odpowiednie hiperparametry, takie jak learning rate, batch size i liczba epok.
- Możesz użyć np. biblioteki `unicodedata` do dekodowania emotikonów w tweetach.

### **2. Trening i ewaluacja:**

- Podziel dane na zbiór treningowy i walidacyjny.
- Wytrenuj model na zbiorze treningowym.
- Zmierz celność na zbiorze walidacyjnym.

### **3. Predykcja na zbiorze testowym:**

- Wykorzystaj wytrenowany model do przypisania autorów tweetów w zbiorze testowym.
- Wyeksportuj predykcje aby były w takim samym formacie jak jest plik test.csv
 


# <h2>Ograniczenia</h2>

- Czas działania twojego kodu(trening i ewaluacja) na T4 na Google Colab powinien wynosić maksymalnie 10 minut.
- Do dyspozycji masz model typu BERT: allegro/herbert-base-cased oraz tokenizer allegro/herbert-base-cased. Nie wolno korzystać z innych uprzednio wytrenowanych modeli oraz ze zbiorów danych innych niż dostarczony.
- Twój kod może trenować się tylko i wyłącznie na danych z pliku train.csv.
- Wszystkie dopuszczalne biblioteki są dostępne w pliku requirements.txt


<h2>Ocenianie</h2>

Ocena zależy od wartości accuracy w sposób liniowy, przy czym:

- Jeśli `accuracy < 0.2`, to liczba punktów wynosi 0.
- Jeśli `accuracy >= 0.7`, to liczba punktów wynosi 1.
- Dla wartości `accuracy` pomiędzy 0.2 a 0.7, liczba punktów rośnie liniowo.

Wzór na obliczenie punktów (P):

$$ P = \frac{accuracy - 0.2}{0.5} $$

Gdzie:
- `accuracy` to wartość dokładności modelu (od 0 do 1).
- Jeśli `accuracy < 0.2`, to P = 0.
- Jeśli `accuracy >= 0.7`, to P = 1.

<h2>Rozwiązanie</h2>

- W tym zadaniu musisz musisz dołączyć plik Zadanie.ipynb, który po włączeniu utworzy plik odpowiedzi.csv gdzie będą znajdowały sie odpowiedzi, w formacie takim jak przykodp.csv, oraz plik odpowiedzi.csv.



In [ ]:
#Jeśli korzystasz z Google Colaba, odkomentuj poniższą linijkę
# !pip install sacremoses
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import random

In [ ]:
MODEL_NAME = 'allegro/herbert-base-cased' 
tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
model = AutoModel.from_pretrained("allegro/herbert-base-cased")

In [ ]:
def set_seed(seed: int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

# Setting a seed for reproducibility
set_seed(42)


In [ ]:
class TwitterDataset(Dataset):
    def __init__(self, messages, labels, tokenizer, max_len):
        self.messages = messages
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.messages)

    def __getitem__(self, index):
        message = str(self.messages[index])
        label = self.labels[index]

        # Tokenize the input
        encoding = self.tokenizer(
            message,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
# Model klasyfikacji na bazie Herberta
class TweetClassifier(nn.Module):
    def __init__(self, base_model_name, num_classes):
        """
        TODO: Zaimplementuj model z Herberta i warstwą klasyfikacji.
        - base_model_name: nazwa modelu (np. allegro/herbert-base-cased).
        - num_classes: liczba klas (5 w tym przypadku).
        """
        super(TweetClassifier, self).__init__()
        # TODO: Załaduj model Herberta i dodaj warstwę klasyfikacyjną.
        pass

    def forward(self, input_ids, attention_mask):
        """
        TODO: Zaimplementuj forward pass modelu.
        - Przyjmij input_ids i attention_mask.
        - Zwróć wyniki klasyfikacji.
        """
        pass

In [ ]:
# Funkcja do trenowania modelu
def train_model(model, data_loader, loss_fn, optimizer, device):
    """
    TODO: Zaimplementuj pętlę treningową.
    - model: model, który trenujesz.
    - data_loader: DataLoader z danymi treningowymi.
    - loss_fn: funkcja straty (np. CrossEntropyLoss).
    - optimizer: optymalizator (np. AdamW).
    - device: urządzenie (CPU/GPU).
    """
    model.train()
    for data in data_loader:
        # TODO: Pobierz input_ids, attention_mask, labels z batcha.
        # TODO: Oblicz predykcje modelu.
        # TODO: Oblicz stratę.
        # TODO: Wykonaj backpropagation i optymalizację.
        pass

def save_predictions(model, data_loader, device, output_file="odpowiedzi.csv"):
    """
    Generuje predykcje na danych testowych i zapisuje je do pliku CSV.
    
    Args:
        test_loader (DataLoader): Dane testowe bez etykiet.
        model (nn.Module): Wytrenowany model.
        device (str): Urządzenie ('cuda' lub 'cpu').
        output_file (str): Nazwa pliku wyjściowego (domyślnie 'odpowiedzi.csv').
    
    TODO:
    1. Wykonaj predykcje dla wszystkich danych w test_loader.
    2. Treści tweetów oraz przewidywane etykiety w formacie CSV.
       - Format CSV: 
         | Content | Label |
    """
    model.eval()
    correct_predictions = 0
    total_samples = 0
    with torch.no_grad():
        for data in data_loader:
            # TODO: Pobierz input_ids, attention_mask, labels z batcha.
            # TODO: Oblicz predykcje modelu.
            # TODO: Policz poprawne predykcje.
            pass
    pass



In [ ]:
def prepare_data(file_path, tokenizer, max_len):
    """
    TODO: Przygotuj dane do treningu i walidacji.
    - file_path: ścieżka do pliku CSV.
    - tokenizer: tokenizer Herberta.
    - max_len: maksymalna długość tokenów.
    """
    # TODO: Wczytaj dane z pliku.
    # TODO: Przekonwertuj etykiety na numeryczne wartości.
    # TODO: Zwróć dataset i DataLoader.
    pass

In [ ]:
def main():
    """
    TODO: Połącz wszystkie komponenty w celu wytrenowania modelu.
    - Przygotuj dane treningowe i walidacyjne.
    - Stwórz model.
    - Wytrenuj model.
    - Oceń model.
    - Zapisz predykcje na zbiorze testowym.
    """
    # Ustawienia
    MODEL_NAME = 'allegro/herbert-base-cased'
    MAX_LEN = 128
    BATCH_SIZE = 16
    NUM_EPOCHS = 5
    NUM_CLASSES = 5
    LEARNING_RATE = 2e-5

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # TODO: Przygotuj tokenizer.
    # TODO: Wczytaj dane i przygotuj DataLoadery.
    # TODO: Stwórz model i przesuń na GPU.
    # TODO: Zdefiniuj funkcję straty i optymalizator.
    # TODO: Trenuj model i zapisuj metryki.

if __name__ == "__main__":
    main()